In [1]:
import pandas as pd
import pulp
import re

In [2]:
salary_cap = 194
roster_spots = {'QB':2
                ,'WR':4
                ,'TE':2
                ,'RB':2
                ,'KR':1
                ,'DE':1
                }

In [3]:
league_df = pd.read_csv('data.csv')

#Get position
league_df['position'] = league_df['Converted'].str[:2]

league_df.head()

,Converted,Points,Exp_Cost,23_Cost,Max_Cost,position
0,QB01,461.60,50,51,51,QB
1,QB02,436.63,47,50,50,QB
2,QB03,413.86,44,49,49,QB
3,QB04,391.19,41,48,48,QB
4,QB05,366.90,34,37,37,QB


In [4]:
salaries = {}
points = {}
for pos in league_df['position'].unique():
    available_pos = league_df[league_df['position']==pos]
    salary = list(available_pos[['Converted','Exp_Cost']].set_index('Converted').to_dict().values())[0]
    point = list(available_pos[['Converted','Points']].set_index('Converted').to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

In [11]:
_vars = {k: pulp.LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [6]:
_vars

{'QB': {'QB01': QB_QB01,
  'QB02': QB_QB02,
  'QB03': QB_QB03,
  'QB04': QB_QB04,
  'QB05': QB_QB05,
  'QB06': QB_QB06,
  'QB07': QB_QB07,
  'QB08': QB_QB08,
  'QB09': QB_QB09,
  'QB10': QB_QB10,
  'QB11': QB_QB11,
  'QB12': QB_QB12,
  'QB13': QB_QB13,
  'QB14': QB_QB14,
  'QB15': QB_QB15,
  'QB16': QB_QB16,
  'QB17': QB_QB17,
  'QB18': QB_QB18,
  'QB19': QB_QB19,
  'QB20': QB_QB20,
  'QB21': QB_QB21,
  'QB22': QB_QB22,
  'QB23': QB_QB23,
  'QB24': QB_QB24,
  'QB25': QB_QB25,
  'QB26': QB_QB26,
  'QB27': QB_QB27,
  'QB28': QB_QB28,
  'QB29': QB_QB29,
  'QB30': QB_QB30,
  'QB31': QB_QB31,
  'QB32': QB_QB32,
  'QB33': QB_QB33,
  'QB34': QB_QB34,
  'QB35': QB_QB35},
 'WR': {'WR01': WR_WR01,
  'WR02': WR_WR02,
  'WR03': WR_WR03,
  'WR04': WR_WR04,
  'WR05': WR_WR05,
  'WR06': WR_WR06,
  'WR07': WR_WR07,
  'WR08': WR_WR08,
  'WR09': WR_WR09,
  'WR10': WR_WR10,
  'WR11': WR_WR11,
  'WR12': WR_WR12,
  'WR13': WR_WR13,
  'WR14': WR_WR14,
  'WR15': WR_WR15,
  'WR16': WR_WR16,
  'WR17': WR_WR17,

In [15]:
_vars.items()

dict_items([('QB', {'QB01': QB_QB01, 'QB02': QB_QB02, 'QB03': QB_QB03, 'QB04': QB_QB04, 'QB05': QB_QB05, 'QB06': QB_QB06, 'QB07': QB_QB07, 'QB08': QB_QB08, 'QB09': QB_QB09, 'QB10': QB_QB10, 'QB11': QB_QB11, 'QB12': QB_QB12, 'QB13': QB_QB13, 'QB14': QB_QB14, 'QB15': QB_QB15, 'QB16': QB_QB16, 'QB17': QB_QB17, 'QB18': QB_QB18, 'QB19': QB_QB19, 'QB20': QB_QB20, 'QB21': QB_QB21, 'QB22': QB_QB22, 'QB23': QB_QB23, 'QB24': QB_QB24, 'QB25': QB_QB25, 'QB26': QB_QB26, 'QB27': QB_QB27, 'QB28': QB_QB28, 'QB29': QB_QB29, 'QB30': QB_QB30, 'QB31': QB_QB31, 'QB32': QB_QB32, 'QB33': QB_QB33, 'QB34': QB_QB34, 'QB35': QB_QB35}), ('WR', {'WR01': WR_WR01, 'WR02': WR_WR02, 'WR03': WR_WR03, 'WR04': WR_WR04, 'WR05': WR_WR05, 'WR06': WR_WR06, 'WR07': WR_WR07, 'WR08': WR_WR08, 'WR09': WR_WR09, 'WR10': WR_WR10, 'WR11': WR_WR11, 'WR12': WR_WR12, 'WR13': WR_WR13, 'WR14': WR_WR14, 'WR15': WR_WR15, 'WR16': WR_WR16, 'WR17': WR_WR17, 'WR18': WR_WR18, 'WR19': WR_WR19, 'WR20': WR_WR20, 'WR21': WR_WR21, 'WR22': WR_WR22, '

In [16]:
prob = pulp.LpProblem("Fantasy", pulp.LpMaximize)
rewards = []
costs = []
position_constraints = []
# Setting up the reward
for k, v in _vars.items():
    costs += pulp.lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += pulp.lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += pulp.lpSum([_vars[k][i] for i in v]) <= roster_spots[k]
    
prob += pulp.lpSum(rewards)
prob += pulp.lpSum(costs) <= salary_cap

In [17]:
prob.solve()

1

In [18]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [19]:
summary(prob)

Variables:

DE_DE01 = 1.0
KR_KR04 = 1.0
QB_QB01 = 1.0
QB_QB02 = 1.0
RB_RB29 = 1.0
RB_RB32 = 1.0
TE_TE09 = 1.0
TE_TE15 = 1.0
WR_WR01 = 1.0
WR_WR28 = 1.0
WR_WR35 = 1.0
WR_WR38 = 1.0
---------------------------------------

Constraints:
5*1.0 + 2*1.0 + 50*1.0 + 47*1.0 + 5*1.0 + 3*1.0 + 10*1.0 + 4*1.0 + 45*1.0 + 10*1.0 + 7*1.0 + 6*1.0 = 194.0
---------------------------------------

Score:
187.33*1.0 + 159.67*1.0 + 461.6*1.0 + 436.63*1.0 + 141.1*1.0 + 129.0*1.0 + 105.17*1.0 + 78.77*1.0 + 316.13*1.0 + 136.04*1.0 + 121.04*1.0 + 116.5*1.0 = 2388.98
